### Construindo um filtro de spam utilizando a regra de Bayes

Neste projeto, eu vou construir um filtro de spam para mensagens SMS usando o algoritmo multinomial Naive Bayes. O objetivo é escrever um programa que classifique novas mensagens com uma precisão superior a 80%. Dessa forma, espero que mais de 80% das novas mensagens sejam classificadas corretamente como spam ou não.

Para treinar o algoritmo, vamos utilizar um conjunto de dados de 5.572 mensagens de SMS que já foram classificadas por humanos. O conjunto de dados foi montado por Tiago A. Almeida e José María Gómez Hidalgo, e pode ser baixado do repositório de Machine Learning da UCI.

### Explorando o conjunto de dados

Vou começar importando a biblioteca Pandas e lendo o dataset.

In [ ]:
import pandas as pd

In [ ]:
sms_spam = pd.read_csv(r'C:/Users/msantos/Downloads/SMSSpamCollection.csv', sep='\t', header=None, names=['Label', 'SMS'])

In [ ]:
print(sms_spam.shape)

In [ ]:
sms_spam.head()

Abaixo, vemos que cerca de 87% das mensagens são de emails que não são spam, ou seja, emails idôneos. Já os 13% restantes são de spam. Esta amostra parece representativa, pois na prática a maioria das mensagens que as pessoas recebem são emails idôneos.

In [ ]:
sms_spam['Label'].value_counts(normalize=True)

### Conjunto de Treinamento e de Testes

Agora vou dividir o nosso conjunto de dados em um conjunto de treinamento e outro de teste, onde o conjunto de treinamento responderá por 80% dos dados e o conjunto de teste pelos 20% restantes.

In [ ]:
# Randomize the dataset
data_randomized = sms_spam.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

Vamos agora analisar o percentual de spam e mensagens idôneas nos conjuntos de treinamento e testes. Esperamos que as porcentagens estejam próximas do que temos no conjunto de dados original, onde cerca de 87% das mensagens são de emails idôneos e os 13% restantes são de spam.

In [ ]:
training_set['Label'].value_counts(normalize=True)

In [ ]:
test_set['Label'].value_counts(normalize=True)

O resultado parece bom. Vamos agora limpar o dataset.

### Limpando o Dataset

Para calcular todas as probabilidades requeridas pelo algoritmo, primeiro precisamos realizar um pouco de limpeza de dados de modo a trazer os dados em um formato que nos permita extrair facilmente todas as informações que precisamos.

Basicamente, queremos transformar os dados da maneira abaixo:

![texte](https://camo.githubusercontent.com/27a4a0a699bd8f0713d73347abe2929c267a03d5/68747470733a2f2f64712d636f6e74656e742e73332e616d617a6f6e6177732e636f6d2f3433332f637067705f646174617365745f332e706e67)

### Letras e Pontuação

Vamos remover toda e qualquer pontuação e transformas todas as letras para minúsculas.

In [ ]:
# Before cleaning
training_set.head()

In [ ]:
# After cleaning
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

### Criação de vocabulário

Passemos agora à criação do vocabulário, que neste contexto significa uma lista com todas as palavras únicas em nosso conjunto de treinamento.

In [ ]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

Parece que existem 7.783 palavras únicas em todas as mensagens do nosso conjunto de treinamento.

In [ ]:
len(vocabulary)

### Conjunto final de treinamento

Vamos agora usar o vocabulário que acabamos de criar para fazer a transformação de dados que queremos.

In [82]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [83]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,reverse,ffffffffff,fancy,independence,compromised,refunded,non,polo,dignity,bet,...,shrek,flat,tbs,punish,meow,poyyarikatur,analysis,2814032,junna,tonsolitusaswell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,reverse,ffffffffff,fancy,independence,compromised,refunded,non,polo,...,shrek,flat,tbs,punish,meow,poyyarikatur,analysis,2814032,junna,tonsolitusaswell
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculando as constantes

Já terminamos de limpar o conjunto de treinamento e podemos começar a criar o filtro de spam. O algoritmo Naive Bayes terá de responder a estas duas questões de probabilidade para poder classificar novas mensagens:

![texte](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)

![texte](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

Além disso, para calcular P(wi|Spam) e P(wi|Ham) dentro das fórmulas acima, vamos precisar usar as equações abaixo:

![texte](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

![texte](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Alguns dos termos nas quatro equações acima terão o mesmo valor para cada nova mensagem. Podemos calcular o valor desses termos uma vez e evitar fazer os cálculos novamente quando uma nova mensagem chegar. Abaixo, vamos usar o nosso conjunto de treinamento para calcular:

- P(Spam) and P(Ham)
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$

Também utilizaremos a suavização de Laplace e configurá-la como a = 1

In [85]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

### Calculando parametros

Agora que temos os termos constantes calculados acima, podemos prosseguir com o cálculo dos parâmetros $P(w_i|Spam)$ e $P(w_i|Ham)$. Cada parâmetro será assim um valor de probabilidade condicional associado a cada palavra do vocabulário.

Os parâmetros são calculados utilizando as fórmulas:

![texte](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

![texte](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

In [86]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

### Classificando uma nova mensagem recebida

Agora que temos todos os nossos parâmetros calculados, podemos começar a criar o filtro de spam. O filtro de spam pode ser entendido como uma função que:

- Recebe como entrada uma nova mensagem (w1, w2, ..., wn).
- Calcula P(Spam|w1, w2, ..., wn) e P(Ham|w1, w2, ..., wn).
- Compara os valores de P(Spam|w1, w2, ..., wn) e P(Ham|w1, w2, ..., wn), e:
    - Se P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), então a mensagem é classificada como presunto.
    - Se P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), então a mensagem é classificada como spam.
    - Se P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), então o algoritmo pode solicitar ajuda humana.

In [87]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [88]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [89]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Medição da Precisão do Filtro de Spam

Os dois resultados acima parecem promissores, mas vamos ver como o filtro se sai bem no nosso conjunto de teste, que tem 1.114 mensagens.

Vamos começar por escrever uma função que devolve as etiquetas de classificação em vez de as imprimir.

In [92]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Agora que temos uma função que devolve etiquetas em vez de as imprimir, podemos usá-la para criar uma nova coluna no nosso conjunto de teste.

In [94]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Agora, vamos escrever uma função para medir a precisão do nosso filtro de spam para descobrir o quão bem o nosso filtro de spam funciona.

In [95]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1100
Incorrect: 14
Accuracy: 0.9874326750448833


A precisão é próxima de 98,74%, o que é realmente bom. Nosso filtro de spam olhou 1.114 mensagens que não viu em treinamento, e classificou 1.100 corretamente.